In [1]:
from dotenv import load_dotenv
import os

# load .env file
load_dotenv("../.env")

# get azure credentials from .env file
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
temperature = os.getenv("TEMPERATURE")

In [2]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from ragas import evaluate

azure_model = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_endpoint=azure_endpoint,
    azure_deployment=deployment_name,
    api_key=api_key,
    temperature=0,
    validate_base_url=False,
)

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
azure_embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=azure_endpoint,
    azure_deployment=embedding_deployment_name,
    api_key=api_key,
    api_version=api_version,
)

D:\Repos\data2day\data2day-rag-workshop\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.document_loaders import DirectoryLoader
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context


loader = DirectoryLoader(
    "../data", use_multithreading=True, silent_errors=True)
documents = loader.load()

for document in documents:
    document.metadata["filename"] = document.metadata["source"]

Error loading file ..\data\AI_auf_den_Markt_bringen_–_Fallstricke_&_Pflichten_.md: 'utf-8' codec can't decode byte 0xe2 in position 46: invalid continuation byte
Error loading file ..\data\Data_Contracts_–_Der_Treiber_für_Automatisierung.md: 'utf-8' codec can't decode byte 0xe2 in position 36: invalid continuation byte
Error loading file ..\data\Bring_your_own_Data_Hands-on_Enterprise_AI_Assistants_in_der_Cloud.md: partition_md() is not available because one or more dependencies are not installed. Use: pip install "unstructured[md]" (including quotes) to install the required dependencies
The MIME type of '..\\data\\Datenanalysen_im_Fußball_–_ein_Überblick.md' is "cannot open `..\\data\\datenanalysen_im_fu\\303\\237ball_\\342\\200\\223_ein_\\303\\234berblick.md' (no such file or directory)". This file type is not currently supported in unstructured.
Error loading file ..\data\Chatte_mit_deinen_Daten_Aufbau_eines_Retrieval-Augmented-Generation-(RAG)-Systems.md: partition_md() is not avai

In [7]:
documents

[Document(metadata={'source': '../data/Road_to_Production_mit_Generative_AI_und_LLMs,_was_hat_sich_geändert_.md', 'filename': '../data/Road_to_Production_mit_Generative_AI_und_LLMs,_was_hat_sich_geändert_.md'}, page_content='Road to Production mit Generative AI und LLMs, was hat sich geändert?\n\nEs hat sich in den letzten zwei Jahren viel getan im Bereich AI und maschinelles Lernen. Large Language Models liefern ganz neue Herangehensweisen, um intelligente Systeme zu bauen. Begannen früher Data- Science-Projekte mit der Datenakquise, so scheint es jetzt zu reichen, den richtigen Prompt zu schreiben.\n\nAber ist das wirklich wahr? Und wie sieht es aus mit der Komplexität, so ein Modell in Produktion zu bringen, wenn man nicht auf existierende SaaS-Angebote zurückgreifen will?\n\nIn diesem Vortrag vergleiche ich "altes ML" mit der neuen Welt im Hinblick auf den Weg in den Live-Betrieb und schaue mir an, was sich geändert hat und was geblieben ist – und ich gebe einen Überblick über das 

In [8]:
generator = TestsetGenerator.from_langchain(
    generator_llm=azure_model, critic_llm=azure_model, embeddings=azure_embeddings
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=10,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

Generating: 100%|██████████| 10/10 [03:02<00:00, 18.29s/it]     


In [9]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,Wie bringt die interne Datenakademie der Merck...,[Wieviel KI müssen wir unseren Kunden erklären...,Die interne Datenakademie der Merck KGaA bring...,simple,[{'source': '../data/Wieviel_KI_müssen_wir_uns...,True
1,Wie hat die Einführung der Transformer-Archite...,[Eine Einführung in Large Language Models\n\nC...,Nach der Einführung der Transformer-Architektu...,simple,[{'source': '../data/Eine_Einführung_in_Large_...,True
2,Wie können Data Scientists realistisches Erwar...,[KI Power-Play: Realistisches Erwartungsmanage...,Data Scientists können realistisches Erwartung...,simple,[{'source': '../data/KI_Power-Play__Realistisc...,True
3,Wie tragen Data-Governance und MLOps-Techniken...,[Die Engineering-Perspektive auf den EU AI Act...,Data-Governance und MLOps-Techniken tragen zur...,simple,[{'source': '../data/Die_Engineering-Perspekti...,True
4,Wie wird die Sentimentanalyse mit LLMs im Work...,[Datenanalyse mit Machine Learning\n\nIn diese...,Die Sentimentanalyse wird im Workshop mit LLMs...,simple,[{'source': '../data/Datenanalyse_mit_Machine_...,True
5,Welche 4 Risikogruppen für KI-Systeme erforder...,[Die Engineering-Perspektive auf den EU AI Act...,Die vier Risikogruppen für KI-Systeme sind ver...,reasoning,[{'source': '../data/Die_Engineering-Perspekti...,True
6,Wie verbessert Christian Winkler die UX in NLP...,[Eine Einführung in Large Language Models\n\nC...,Christian Winkler konzentriert sich auf die Op...,reasoning,[{'source': '../data/Eine_Einführung_in_Large_...,True
7,How do data availability and explainable AI im...,[Datenanalysen im Fußball – ein Überblick\n\nS...,The context discusses the impact of data avail...,multi_context,[{'source': '../data/Datenanalysen_im_Fußball_...,True
8,How has Kira Engelhardt's finance background a...,[KI Power-Play: Realistisches Erwartungsmanage...,Kira Engelhardt's finance background and leade...,multi_context,[{'source': '../data/KI_Power-Play__Realistisc...,True
9,Welche technischen Anforderungen gibt es für T...,[Datenanalyse mit Machine Learning\n\nIn diese...,"Die technischen Anforderungen für Teilnehmer, ...",simple,[{'source': '../data/Datenanalyse_mit_Machine_...,True
